In [ ]:
import pandas as pd

def generate_config():
    all_classes = pd.read_csv('images/train_labels.csv')['class'].unique()
    test_example_count = len(pd.read_csv('images/test_labels.csv'))-1
    print('Done: CSV parsed Unique classes: ',all_classes)

    num_steps = 15000

    generate_tf_record = ''

    f = open("training/labelmap.pbtxt", "w")
    for idx, value in enumerate(all_classes):

        first_class = '''    
        if row_label == 'first':
            return 1
                '''
        all_other_class = '''
        elif row_label == 'other':
            return 2
        '''    
        label_map = '''
    item {
      id: 1
      name: first
    }
    '''

        if idx == 0:
            to_append = first_class.replace('first',value)
        else:
            to_append = all_other_class.replace('other',value).replace('2',str(idx+1))

        generate_tf_record+=to_append

        f.write(label_map.replace('first',value).replace('1',str(idx+1)))

    f.close()

    #print('\nGenerated string:\n',generate_tf_record)

    with open("generate_tfrecord_base.py") as f:
        lines = f.readlines()

    with open("generate_tfrecord.py", "w") as f:
        lines.insert(32, generate_tf_record)
        f.write("".join(lines))

    print('Done: Edited generate_tfrecord.py')
    print('Done: Edited labelmap.pbtxt')

    with open("training/faster_rcnn_inception_v2_pets_base.config") as f:
        lines = f.readlines()

    with open("training/faster_rcnn_inception_v2_pets.config", "w") as f:
        lines[8] = '    num_classes: {}\n'.format(str(len(all_classes)))
        lines[131] = '  num_examples: {}\n'.format(str(test_example_count))
        lines[115]= '  num_steps: {}\n'.format(str(num_steps))
        f.write("".join(lines))
        
    print('Done: Edited training/faster_rcnn_inception_v2_pets.config')

In [ ]:
generate_config()

In [1]:
from static_files.etc_utils import generate_config, do_xml_to_csv, generate_tfrecords
import os

In [3]:
base_config_path = '/home/aniruddh/lincode/product/TensorFlow-Object-Detection-API/'
to_save_path = '/home/aniruddh/lincode/product/livis-develop_v2/experiments/tensorflow/t2/'
dataset_path = '/home/aniruddh/lincode/product/images/'
num_steps = 15000

os.makedirs(to_save_path, exist_ok=True)

do_xml_to_csv(to_save_path, dataset_path)
config_path = generate_config(base_config_path, to_save_path, num_steps)
generate_tfrecords(base_config_path, to_save_path, dataset_path)

train_tf_model(base_config_path, to_save_path, dataset_path, num_steps)

Done: CSV parsed Unique classes:  ['queen' 'ten' 'ace' 'king' 'nine' 'jack']
Done: Edited generate_tfrecord.py
Done: Edited labelmap.pbtxt
Done: Edited training/faster_rcnn_inception_v2_pets.config
Done: TF Records


In [ ]:
!ls /home/aniruddh/lincode/product/livis-develop_v2/experiments/tensorflow/ex3_data_path/data/